# IWTC Tools Environment Setup
Environment configuration and reproducibility notebook for IWTC-Tools.

This notebook documents and automates environment creation for the IWTC project.
It assumes familiarity with Python, venv, pip, and JupyterLab.

## 1. Verify base Python installation

In [11]:
!python3.11 --version

Python 3.11.14


## 2. Create or refresh the virtual environment

In [13]:
!/usr/local/opt/python@3.11/bin/python3.11 -m venv .venv

## 3. Activate and verify the environment (Terminal step)
```bash
cd $HOME/iwtc-tools
source .venv/bin/activate
python --version
```

## 4. Install core packages
Installs required dependencies for notebooks and IWTC-Lab utilities.
Package rationale:
- pip, setuptools, wheel — modern packaging/installer toolchain.
- notebook, jupyterlab — Jupyter frontends (Lab 4 preferred).
- ipywidgets — interactive controls for notebook UIs.
- pandas — tabular logs, initiative/state tables, summaries.
- Pillow — image handling for maps, tokens, thumbnails.
- ruamel.yaml — robust YAML (round-tripping and comments preserved).
- requests — lightweight HTTP for fetching reference data or assets.

In [14]:
import sys, subprocess
packages = [
    "pip","setuptools","wheel",
    "notebook","jupyterlab","ipywidgets",
    "pandas","Pillow","ruamel.yaml","requests",
]
for pkg in packages:
    print(f"Installing/upgrading {pkg} ...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", pkg])
print("\nCore IWTC-Lab environment ready.")

Installing/upgrading pip ...
Installing/upgrading setuptools ...
Installing/upgrading wheel ...
Installing/upgrading notebook ...
Installing/upgrading jupyterlab ...
Installing/upgrading ipywidgets ...
Installing/upgrading pandas ...
Installing/upgrading Pillow ...
Installing/upgrading ruamel.yaml ...
Installing/upgrading requests ...

Core IWTC-Lab environment ready.


## 5. Configure PYTHONPATH for project imports

In [16]:
import os
from pathlib import Path

venv = os.environ.get("VIRTUAL_ENV")
if not venv:
    print("VIRTUAL_ENV is not set. Start JupyterLab from an activated venv first.")
else:
    activate = Path(venv) / "bin" / "activate"
    project_root = Path(venv).parent.resolve()  # iwtc-lab/
    export_line = f'export PYTHONPATH="{project_root}:$PYTHONPATH"\n'

    if not activate.exists():
        print(f"Activate script not found at: {activate}")
    else:
        txt = activate.read_text()
        if export_line not in txt:
            if not txt.endswith("\n"):
                txt += "\n"
            activate.write_text(txt + export_line)
            print(f"Added PYTHONPATH export to {activate}")
        else:
            print("PYTHONPATH export already present.")

    print("Project root:", project_root)
    print("Activate path:", activate)


Added PYTHONPATH export to /Users/charissophia/iwtc-lab/.venv/bin/activate
Project root: /Users/charissophia/iwtc-lab
Activate path: /Users/charissophia/iwtc-lab/.venv/bin/activate


## 6. Verify configuration from within Jupyter

In [4]:
import sys, os, platform
print("Python executable:", sys.executable)
print("Python version:", sys.version.split()[0])
print("Platform:", platform.platform())
print("PYTHONPATH:", os.environ.get("PYTHONPATH"))

Python executable: /Users/charissophia/iwtc-lab/.venv/bin/python
Python version: 3.11.14
Platform: macOS-11.7.10-x86_64-i386-64bit
PYTHONPATH: ~/iwtc-lab:


## 7. Ensure project structure exists

In [5]:
from pathlib import Path
root = Path.cwd()
folders = {
    "assets/maps":"Maps","assets/tokens":"Tokens","assets/icons":"Icons",
    "data/monsters":"Monsters","data/npcs":"NPCs","data/encounters":"Encounters",
    "lib":"Library","notebooks":"Notebooks","sessions":"Sessions"
}
for p in folders: (root/p).mkdir(parents=True, exist_ok=True)
for p, why in folders.items(): print(f"• {p:<20} — {why}")

• assets/maps          — Maps
• assets/tokens        — Tokens
• assets/icons         — Icons
• data/monsters        — Monsters
• data/npcs            — NPCs
• data/encounters      — Encounters
• lib                  — Library
• notebooks            — Notebooks
• sessions             — Sessions


## 8. Export dependency list

In [1]:
!pip freeze --local > requirements.txt
!head -n 25 requirements.txt

anyio==4.11.0
appnope==0.1.4
argon2-cffi==25.1.0
argon2-cffi-bindings==25.1.0
arrow==1.4.0
asttokens==3.0.0
async-lru==2.0.5
attrs==25.4.0
babel==2.17.0
beautifulsoup4==4.14.2
bleach==6.3.0
certifi==2025.10.5
cffi==2.0.0
charset-normalizer==3.4.4
comm==0.2.3
debugpy==1.8.17
decorator==5.2.1
defusedxml==0.7.1
executing==2.2.1
fastjsonschema==2.21.2
fqdn==1.5.1
h11==0.16.0
httpcore==1.0.9
httpx==0.28.1
idna==3.11


## 9. Quick import test

In [1]:
import sys
print("Project root on sys.path?", "$HOME/iwtc-lab" in sys.path)
try:
    from lib import dice
    print("Import lib.dice: OK")
except Exception as e:
    print("Import lib.dice: FAILED ->", e)

Project root on sys.path? False
Import lib.dice: OK


## 10. Environment summary

In [2]:
import datetime, os, sys, subprocess
print("IWTC-Lab environment summary —", datetime.datetime.now().isoformat())
print("Python executable:", sys.executable)
print("Python version:", sys.version.split()[0])
print("PYTHONPATH:", os.environ.get("PYTHONPATH"))
print("\nKey packages:")
for p in ["jupyterlab","notebook","ipywidgets","pandas","Pillow","ruamel.yaml","requests"]:
    try:
        v = subprocess.check_output([sys.executable,"-m","pip","show",p],text=True)
        ver = [line.split(":",1)[1].strip() for line in v.splitlines() if line.startswith("Version:")]
        print(f" - {p:<12} {ver[0] if ver else '(unknown)'}")
    except Exception:
        print(f" - {p:<12} (not installed)")

IWTC-Lab environment summary — 2025-10-27T18:38:37.630283
Python executable: /Users/charissophia/iwtc-lab/.venv/bin/python
Python version: 3.11.14
PYTHONPATH: ~/iwtc-lab:

Key packages:
 - jupyterlab   4.4.10
 - notebook     7.4.7
 - ipywidgets   8.1.7
 - pandas       2.3.3
 - Pillow       12.0.0
 - ruamel.yaml  0.18.16
 - requests     2.32.5


## 11. Maintenance notes
- Re-run package install and freeze after adding packages.
- `.venv` is excluded from version control.
- After recreating `.venv`, re-run cell 5 to append the PYTHONPATH line.
- Always launch JupyterLab from the activated venv: `python -m jupyter lab`.